La tabla "tbl_flujo_reporte" debe resumir en cada "id_item" el contenido del campo NETO de la tabla "tbl_transacciones" según "ITEM_COD", "ANIO" y "MES"

Evaluar las siguientes opciones:
1. ir creando una columna en tabla "tbl_flujo_reporte" para resumir la información de cada mes/año
2. Modificar la estructura de la tabla "tbl_flujo_reporte" para que reciba un registro por cada acumulado de NETO por "ITEM_COD",   "ANIO" y "MES"

========================================================================================

1. **Crear una columna en la tabla "tbl_flujo_reporte" para resumir la información de cada mes/año**: 
* Esta opción implica agregar una nueva columna para cada mes/año en la tabla "tbl_flujo_reporte". Cada vez que se ingresen nuevos datos en la tabla "tbl_transacciones", tendrás que actualizar las columnas correspondientes en "tbl_flujo_reporte". 
* Esto podría ser eficiente si necesitas acceder rápidamente a los totales mensuales/año sin tener que realizar cálculos complejos en tiempo de consulta. 
* Sin embargo, podría volverse complicado si hay muchos meses/años diferentes o si la cantidad de datos es grande, ya que tendrías muchas columnas en la tabla "tbl_flujo_reporte".

2. **Modificar la estructura de la tabla "tbl_flujo_reporte" para que reciba un registro por cada acumulado de NETO por "ITEM_COD", "ANIO" y "MES"**: 
* En esta opción, en lugar de tener una columna para cada mes/año en "tbl_flujo_reporte", tendrías una fila por cada combinación de "ITEM_COD", "ANIO" y "MES", con una columna que almacena el total NETO correspondiente. 
* Esto sería más escalable y flexible, ya que no tendrías que modificar la estructura de la tabla cada vez que agregues un nuevo mes/año. 
* Además, permitiría consultas más flexibles y eficientes sobre los datos agregados.

En general, la opción 2 parece ser la más flexible y escalable, ya que te permite almacenar los totales NETO de manera estructurada y realizar consultas eficientes sobre ellos. Sin embargo, la elección final dependerá de tus necesidades específicas y de la complejidad de tu aplicación.

========================================================================================

Uno de los propósitos de la tabla "tbl_flujo_reporte" es que sirva como puente para tomar datos que alimentarán un reporte

* Si el propósito principal de la tabla "tbl_flujo_reporte" es alimentar un reporte, entonces la opción 2 parece ser la más adecuada. 
* Al tener una fila por cada combinación de "ITEM_COD", "ANIO" y "MES" con el total NETO correspondiente, puedes generar fácilmente los datos necesarios para el reporte utilizando consultas SQL simples.
* Además, al utilizar esta estructura, la tabla "tbl_flujo_reporte" actuará como un puente eficiente para obtener los datos agregados necesarios para el reporte, ya que los totales NETO estarán precalculados y disponibles en la tabla, lo que evitará la necesidad de realizar cálculos complejos en tiempo de consulta.

Por lo tanto, la opción 2 te proporcionará una estructura eficiente y escalable para alimentar el reporte, lo que facilitará el proceso de generación del mismo y garantizará un rendimiento óptimo, especialmente si estás trabajando con grandes volúmenes de datos.

========================================================================================

In [1]:
import pandas as pd

import mysql.connector
import configparser
from openpyxl import Workbook

import openpyxl
from openpyxl.utils import get_column_letter
from openpyxl.styles import Font

In [2]:
# ============================================================================================================
# Crear un dataframe con los datos de la tabla tbl_flujo_reporte según rango definido en parámetros de entrada
# ============================================================================================================

def report_mes(anio_inicio, mes_inicio, anio_fin, mes_fin):

    # Leer las credenciales del archivo de configuración
    config = configparser.ConfigParser()
    config.read('config.ini')

    user = config['database']['user']
    password = config['database']['password']
    host = config['database']['host']
    database = config['database']['database']

    # Abrir la base de datos: establecer la conexión
    conexion = mysql.connector.connect(user=user, password=password, host=host, database=database)

    # ============================================================================================================

    # Consulta SQL para extraer datos de tbl_flujo_reporte
    query = f"""
        SELECT *
        FROM tbl_flujo_reporte
        WHERE (anio > {anio_inicio} OR (anio = {anio_inicio} AND mes >= {mes_inicio}))
        AND (anio < {anio_fin} OR (anio = {anio_fin} AND mes <= {mes_fin}))
    """

    # Ejecutar la consulta y cargar los datos en un DataFrame
    reporte_mes = pd.read_sql(query, conexion)

    # Cerrar la conexión a la base de datos
    conexion.close()

    return reporte_mes

# ============================================================================================================

# Crear un dataframe con los datos de la tabla tbl_flujo_reporte según rango definido en parámetros de entrada
# Ejemplo de uso:
reporte_mes = report_mes(2023, 1, 2023, 2)
# print(reporte_mes)

# Crea la columna 'periodo' concatenando las columnas 'anio', 'mes'
reporte_mes['periodo'] = pd.to_datetime(reporte_mes['anio'].astype(str) + '-' + reporte_mes['mes'].astype(str) )
reporte_mes.tail()

C:\Users\gusta\AppData\Local\Temp\ipykernel_12840\3356292400.py:30: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  reporte_mes = pd.read_sql(query, conexion)


,id_item,id_item_cod,anio,mes,total_neto,periodo
33,1306,29,2023,1,28749303.00,2023-01-01
34,1307,32,2023,1,510629.00,2023-01-01
35,1308,33,2023,1,2712000.00,2023-01-01
36,1309,35,2023,1,1163000.00,2023-01-01
37,1310,38,2023,1,164126.98,2023-01-01


In [3]:
reporte_mes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38 entries, 0 to 37
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   id_item      38 non-null     int64         
 1   id_item_cod  38 non-null     int64         
 2   anio         38 non-null     int64         
 3   mes          38 non-null     int64         
 4   total_neto   38 non-null     float64       
 5   periodo      38 non-null     datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(4)
memory usage: 1.9 KB


In [4]:
# Crear el DataFrame reporte_final utilizando pivot_table
reporte_final = pd.pivot_table(reporte_mes, 
                               values='total_neto', 
                               index='id_item_cod', 
                               columns='periodo', 
                               aggfunc='sum')

# Conservar el formato datetime en los títulos de las columnas
reporte_final.columns = reporte_final.columns.strftime('%Y-%m')


reporte_final.reset_index(inplace = True)

# Mostrando el nuevo dataframe
reporte_final

periodo,id_item_cod,2023-01,2023-02
0,2,330000.00,330000.00
1,3,393648.86,456206.92
2,5,NaN,37000.00
3,7,2220953.00,3558973.08
4,8,2350098.00,4631322.00
5,9,-39685569.00,-45085363.00
6,11,3338960.00,3826570.00
7,12,115973.92,112080.15
8,14,-2898.16,-2323.64
9,15,5632376.00,5281700.00


In [5]:
reporte_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id_item_cod  22 non-null     int64  
 1   2023-01      18 non-null     float64
 2   2023-02      20 non-null     float64
dtypes: float64(2), int64(1)
memory usage: 656.0 bytes


In [6]:
# ============================================================================================================
# Crear un dataframe con los datos de la tabla tbl_fcdir_items
# ============================================================================================================

def data_items():

    # Leer las credenciales del archivo de configuración
    config = configparser.ConfigParser()
    config.read('config.ini')

    user = config['database']['user']
    password = config['database']['password']
    host = config['database']['host']
    database = config['database']['database']

    # Abrir la base de datos: establecer la conexión
    conexion = mysql.connector.connect(user=user, password=password, host=host, database=database)

    # ============================================================================================================

    # Consulta SQL para extraer datos de tbl_fcdir_items
    query = f"""
        SELECT *
        FROM tbl_fcdir_items
    """

    # Ejecutar la consulta y cargar los datos en un DataFrame
    tabla_items = pd.read_sql(query, conexion)

    # Cerrar la conexión a la base de datos
    conexion.close()

    return tabla_items

# ============================================================================================================

# Crear un dataframe con los datos de la tabla tbl_flujo_reporte según rango definido en parámetros de entrada
# Ejemplo de uso:
tabla_items = data_items()
tabla_items.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id_item  45 non-null     int64 
 1   name     45 non-null     object
 2   id_grp   45 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 1.2+ KB


C:\Users\gusta\AppData\Local\Temp\ipykernel_12840\4204913904.py:28: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tabla_items = pd.read_sql(query, conexion)


In [7]:
# ============================================================================================================
# Crear un dataframe con los datos de la tabla tbl_fcdir_grupos
# ============================================================================================================

def data_grupos():

    # Leer las credenciales del archivo de configuración
    config = configparser.ConfigParser()
    config.read('config.ini')

    user = config['database']['user']
    password = config['database']['password']
    host = config['database']['host']
    database = config['database']['database']

    # Abrir la base de datos: establecer la conexión
    conexion = mysql.connector.connect(user=user, password=password, host=host, database=database)

    # ============================================================================================================

    # Consulta SQL para extraer datos de tbl_fcdir_grupos
    query = f"""
        SELECT *
        FROM tbl_fcdir_grupos
    """

    # Ejecutar la consulta y cargar los datos en un DataFrame
    tabla_grupos = pd.read_sql(query, conexion)

    # Cerrar la conexión a la base de datos
    conexion.close()

    return tabla_grupos

# ============================================================================================================

# Crear un dataframe con los datos de la tabla tbl_flujo_reporte según rango definido en parámetros de entrada
# Ejemplo de uso:
tabla_grupos = data_grupos()
tabla_grupos.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id_grp  6 non-null      int64 
 1   name    6 non-null      object
dtypes: int64(1), object(1)
memory usage: 224.0+ bytes


C:\Users\gusta\AppData\Local\Temp\ipykernel_12840\812418555.py:28: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tabla_grupos = pd.read_sql(query, conexion)


In [8]:
# ============================================================================================================
# Actualiza 'reporte_' con datos de 'tabla_items'
# ============================================================================================================

# Realizar la fusión basada en la columna 'id_item_cod' de la tabla 'reporte_mes' y 'id_item' de la tabla 'tabla_items'
reporte_final = pd.merge(reporte_final, tabla_items, left_on='id_item_cod', right_on='id_item', how='left')

# Eliminar la columna 'id_item' ya que se ha utilizado para hacer la fusión y ahora está duplicada
# reporte_final.drop('id_item_y', axis=1, inplace=True)

# Mostrar el DataFrame resultante con la información agregada
reporte_final.head()


,id_item_cod,2023-01,2023-02,id_item,name,id_grp
0,2,330000.00,330000.00,2,Accounting software,4
1,3,393648.86,456206.92,3,Bank comissions & related,4
2,5,NaN,37000.00,5,Certificates,4
3,7,2220953.00,3558973.08,7,Cleaning service,4
4,8,2350098.00,4631322.00,8,Cleaning supplies,4


In [9]:
reporte_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id_item_cod  22 non-null     int64  
 1   2023-01      18 non-null     float64
 2   2023-02      20 non-null     float64
 3   id_item      22 non-null     int64  
 4   name         22 non-null     object 
 5   id_grp       22 non-null     int64  
dtypes: float64(2), int64(3), object(1)
memory usage: 1.2+ KB


In [10]:
# ============================================================================================================
# Actualiza 'reporte_final' con datos de 'tabla_grupos'
# ============================================================================================================


# Realizar la fusión basada en la columna 'id_grp' de la tabla 'reporte_final' con la columna 'id_grp' de la tabla 'tabla_grupos'
reporte_final = pd.merge(reporte_final, tabla_grupos, left_on='id_grp', right_on='id_grp', how='left')

reporte_final

,id_item_cod,2023-01,2023-02,id_item,name_x,id_grp,name_y
0,2,330000.00,330000.00,2,Accounting software,4,OPERATING
1,3,393648.86,456206.92,3,Bank comissions & related,4,OPERATING
2,5,NaN,37000.00,5,Certificates,4,OPERATING
3,7,2220953.00,3558973.08,7,Cleaning service,4,OPERATING
4,8,2350098.00,4631322.00,8,Cleaning supplies,4,OPERATING
5,9,-39685569.00,-45085363.00,9,Collect from clients,4,OPERATING
6,11,3338960.00,3826570.00,11,Facility: Electricity,4,OPERATING
7,12,115973.92,112080.15,12,Hosting & domains,4,OPERATING
8,14,-2898.16,-2323.64,14,Interests received,2,FINANCING
9,15,5632376.00,5281700.00,15,Internet,4,OPERATING


In [11]:
reporte_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id_item_cod  22 non-null     int64  
 1   2023-01      18 non-null     float64
 2   2023-02      20 non-null     float64
 3   id_item      22 non-null     int64  
 4   name_x       22 non-null     object 
 5   id_grp       22 non-null     int64  
 6   name_y       22 non-null     object 
dtypes: float64(2), int64(3), object(2)
memory usage: 1.3+ KB


In [12]:
reporte_final.drop(columns=['id_item'], inplace=True)
reporte_final.rename(columns={'name_x': 'Concepto'}, inplace=True)
reporte_final.rename(columns={'name_y': 'Grupo'}, inplace=True)

reporte_final.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id_item_cod  22 non-null     int64  
 1   2023-01      18 non-null     float64
 2   2023-02      20 non-null     float64
 3   Concepto     22 non-null     object 
 4   id_grp       22 non-null     int64  
 5   Grupo        22 non-null     object 
dtypes: float64(2), int64(2), object(2)
memory usage: 1.2+ KB


In [13]:

# Extraer las columnas 'Concepto', 'id_grp' y 'Grupo'
concepto_col = reporte_final.pop('Concepto')
id_grp_col = reporte_final.pop('id_grp')
grupo_col = reporte_final.pop('Grupo')

# Insertar las columnas en la nueva ubicación
reporte_final.insert(1, 'Concepto', concepto_col)
reporte_final.insert(2, 'id_grp', id_grp_col)
reporte_final.insert(3, 'Grupo', grupo_col)

reporte_final


,id_item_cod,Concepto,id_grp,Grupo,2023-01,2023-02
0,2,Accounting software,4,OPERATING,330000.00,330000.00
1,3,Bank comissions & related,4,OPERATING,393648.86,456206.92
2,5,Certificates,4,OPERATING,NaN,37000.00
3,7,Cleaning service,4,OPERATING,2220953.00,3558973.08
4,8,Cleaning supplies,4,OPERATING,2350098.00,4631322.00
5,9,Collect from clients,4,OPERATING,-39685569.00,-45085363.00
6,11,Facility: Electricity,4,OPERATING,3338960.00,3826570.00
7,12,Hosting & domains,4,OPERATING,115973.92,112080.15
8,14,Interests received,2,FINANCING,-2898.16,-2323.64
9,15,Internet,4,OPERATING,5632376.00,5281700.00


In [14]:
# Filtrar las filas donde el valor de 'id_grp' es igual a 1 y eliminarlas
reporte_final = reporte_final.drop(reporte_final[reporte_final['id_grp'] == 1].index)

# Mostrar el DataFrame actualizado
reporte_final

,id_item_cod,Concepto,id_grp,Grupo,2023-01,2023-02
0,2,Accounting software,4,OPERATING,330000.00,330000.00
1,3,Bank comissions & related,4,OPERATING,393648.86,456206.92
2,5,Certificates,4,OPERATING,NaN,37000.00
3,7,Cleaning service,4,OPERATING,2220953.00,3558973.08
4,8,Cleaning supplies,4,OPERATING,2350098.00,4631322.00
5,9,Collect from clients,4,OPERATING,-39685569.00,-45085363.00
6,11,Facility: Electricity,4,OPERATING,3338960.00,3826570.00
7,12,Hosting & domains,4,OPERATING,115973.92,112080.15
8,14,Interests received,2,FINANCING,-2898.16,-2323.64
9,15,Internet,4,OPERATING,5632376.00,5281700.00
